#  Apercu Initial des Donnees  Morocco Student Data Pool

Ce notebook effectue un **premier apercu rapide** du dataset brut avant le nettoyage.

**Objectif :** Comprendre la structure des donnees, identifier les problemes a traiter dans le nettoyage.

### Pipeline du projet :
1. **`analyze_data.ipynb`**  (ce notebook) Apercu initial des donnees brutes
2. **`clean_data.ipynb`**  Nettoyage et preparation des donnees
3. **`Advanced_EDA_Morocco_Students.ipynb`**  Analyse exploratoire avancee sur les donnees nettoyees

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(" Bibliotheques importees")

 Bibliotheques importees


## 1. Chargement et structure du dataset

In [2]:
# Chargement du dataset brut
df = pd.read_csv('../dataset/Morocco_Student_Data_Pool.csv', low_memory=False)

print(f" Dataset charge avec succes")
print(f"   Lignes  : {df.shape[0]:,}")
print(f"   Colonnes: {df.shape[1]}")
print(f"\n Types de donnees :")
print(df.dtypes.value_counts().to_string())

 Dataset charge avec succes
   Lignes  : 10,000
   Colonnes: 286

 Types de donnees :
str        173
float64     65
int64       48


## 1b. Correction du decalage de colonnes

Le CSV brut contient un **decalage d'une colonne** vers la gauche dans les dernieres colonnes (apres `performance_cible`).  
Par exemple, `date_collecte` contient en realite les codes collecteurs (`COL008`), et les vraies dates se trouvent dans `intervention_necessaire`.  
On corrige ici avant toute analyse.

In [3]:
shifted_cols = [
    'probabilite_reussite',    # contient  niveau_risque
    'niveau_risque',           # contient  intervention_necessaire
    'intervention_necessaire', # contient  date_collecte
    'date_collecte',           # contient  id_collecteur
    'id_collecteur',           # contient  statut_verification
    'statut_verification',     # contient  date_mise_a_jour
]

cols_exist = all(c in df.columns for c in shifted_cols)
if cols_exist and str(df['date_collecte'].iloc[0]).startswith('COL'):
    print(" Decalage de colonnes detecte ! Correction en cours...")
    
    saved = {col: df[col].copy() for col in shifted_cols}
    
    df['niveau_risque'] = saved['probabilite_reussite']
    df['intervention_necessaire'] = saved['niveau_risque']
    df['date_collecte'] = saved['intervention_necessaire']
    df['id_collecteur'] = saved['date_collecte']
    df['statut_verification'] = saved['id_collecteur']
    df['probabilite_reussite'] = np.nan
    
    print(" Colonnes realignees :")
    print(f"   date_collecte[0] = {df['date_collecte'].iloc[0]}")
    print(f"   id_collecteur[0] = {df['id_collecteur'].iloc[0]}")
    print(f"   intervention_necessaire[0] = {df['intervention_necessaire'].iloc[0]}")
else:
    print(" Pas de decalage detecte  colonnes correctes.")

 Decalage de colonnes detecte ! Correction en cours...
 Colonnes realignees :
   date_collecte[0] = 2026-01-15
   id_collecteur[0] = COL008
   intervention_necessaire[0] = Oui


In [4]:
# Apercu des premieres lignes
df.head(5)

,id_etudiant,prenom,nom,nom_complet,sexe,date_naissance,age,code_massar,region,province,...,francais_maison,performance_cible,probabilite_reussite,niveau_risque,intervention_necessaire,date_collecte,id_collecteur,statut_verification,date_mise_a_jour,remarques
0,STU00001,Soumia,Chraibi,Soumia Chraibi,F,2007-05-05,19,G548679848,Fes-Meknes,El Hajeb,...,Tres Eleve,0.54,NaN,Moyen,Oui,2026-01-15,COL008,Verifie,NaN,NaN
1,STU00002,Mehdi,Tazi,Mehdi Tazi,M,2009-10-12,17,G478451439,Souss-Massa,Inezgane,...,Eleve,0.08,NaN,Eleve,Oui,2026-01-15,COL005,Verifie,NaN,NaN
2,STU00003,Bilal,Lahbabi,Bilal Lahbabi,M,2008-08-08,18,G727518594,Casablanca-Settat,Berrechid,...,Tres Eleve,0.47,NaN,Eleve,Oui,2026-01-15,COL001,Verifie,NaN,NaN
3,STU00004,Hakim,Chraibi,Hakim Chraibi,M,2009-10-05,17,G868905524,Tanger-Tetouan-Al Hoceima,Larache,...,Eleve,0.55,NaN,Moyen,Oui,2026-01-15,COL008,Verifie,NaN,NaN
4,STU00005,Ayoub,Lahbabi,Ayoub Lahbabi,M,2007-05-27,19,G809425460,Casablanca-Settat,El Jadida,...,Eleve,0.15,NaN,Eleve,Oui,2026-01-15,COL004,Verifie,NaN,NaN


In [5]:
# Informations detaillees
df.info(verbose=False, show_counts=True)

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 286 entries, id_etudiant to remarques
dtypes: float64(66), int64(48), str(172)
memory usage: 32.2 MB


## 2. Valeurs manquantes

Identification des colonnes avec des valeurs manquantes pour planifier la strategie de nettoyage.

In [6]:
# Tableau des valeurs manquantes
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(1)

missing_df = pd.DataFrame({
    'Valeurs manquantes': missing,
    'Pourcentage (%)': missing_pct,
    'Type': [str(df[col].dtype) for col in missing.index]
})

print(f" {len(missing)} colonnes avec des valeurs manquantes sur {df.shape[1]} colonnes totales")
print(f"   Total NaN : {df.isnull().sum().sum():,} ({df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100:.2f}%)\n")
print(missing_df.to_string())

 27 colonnes avec des valeurs manquantes sur 286 colonnes totales


   Total NaN : 214,699 (7.51%)

                         Valeurs manquantes  Pourcentage (%)     Type
remarques                             10000            100.0  float64
note_examen_national                  10000            100.0  float64
note_controle_continu                 10000            100.0  float64
note_finale_bac                       10000            100.0  float64
mention_bac                           10000            100.0  float64
probabilite_reussite                  10000            100.0  float64
date_mise_a_jour                      10000            100.0  float64
gestion_annuel                        10000            100.0  float64
gestion_s2                            10000            100.0  float64
gestion_s1                            10000            100.0  float64
comptabilite_annuel                   10000            100.0  float64
comptabilite_s2                       10000            100.0  float64
comptabilite_s1                       10000            100

## 3. Doublons

In [7]:
# Verification des doublons
n_duplicates = df.duplicated().sum()
print(f" Nombre de lignes dupliquees : {n_duplicates}")

if 'id_etudiant' in df.columns:
    n_dup_id = df['id_etudiant'].duplicated().sum()
    print(f"   IDs etudiants dupliques : {n_dup_id}")

if n_duplicates == 0:
    print("    Aucun doublon detecte")

 Nombre de lignes dupliquees : 0
   IDs etudiants dupliques : 0
    Aucun doublon detecte


## 4. Valeurs uniques  Variables categorielles cles

In [8]:
# Variables categorielles principales
cat_cols = ['sexe', 'region', 'filiere', 'niveau_scolaire', 'milieu',
            'type_etablissement', 'niveau_pere', 'niveau_mere']
cat_cols = [c for c in cat_cols if c in df.columns]

print(f" Valeurs uniques par colonne categorielle :\n")
for col in cat_cols:
    n_unique = df[col].nunique()
    top_val = df[col].value_counts().index[0]
    top_freq = df[col].value_counts().values[0]
    print(f"  {col:<25}  {n_unique:>3} valeurs uniques  (top: '{top_val}' = {top_freq:,})")

 Valeurs uniques par colonne categorielle :

  sexe                         2 valeurs uniques  (top: 'F' = 5,017)
  region                      12 valeurs uniques  (top: 'Beni Mellal-Khenifra' = 952)
  filiere                      7 valeurs uniques  (top: 'Sciences et Technologies' = 2,212)
  type_etablissement           1 valeurs uniques  (top: 'Lycee Qualifiant' = 10,000)


## 5. Statistiques descriptives rapides

In [9]:
# Statistiques descriptives pour les variables numeriques cles
num_cols_key = ['age', 'revenu_mensuel_pere', 'revenu_mensuel_mere', 'revenu_familial',
                'nombre_freres_soeurs', 'taux_assiduite', 'taux_ponctualite',
                'taux_remise_devoirs', 'performance_cible']
num_cols_key = [c for c in num_cols_key if c in df.columns]

print(" Statistiques descriptives  Variables cles :\n")
print(df[num_cols_key].describe().round(2).to_string())

 Statistiques descriptives  Variables cles :



            age  revenu_mensuel_pere  revenu_mensuel_mere  revenu_familial  nombre_freres_soeurs  taux_assiduite  taux_ponctualite  taux_remise_devoirs  performance_cible
count  10000.00             10000.00             10000.00         10000.00               10000.0        10000.00          10000.00             10000.00           10000.00
mean      18.09              9009.12              5823.90         14020.34                   2.5           89.22             91.29                82.90               0.41
std        0.79              5966.74              4600.93          7406.40                   1.4            6.54              5.39                10.15               0.19
min       17.00                 0.00                 0.00             0.00                   0.0           78.00             82.00                65.00               0.08
25%       17.00              4613.75              2643.00          8567.75                   2.0           84.00             87.00               

## 6. Diagnostic pour le nettoyage

Resume des problemes identifies a traiter dans `clean_data.ipynb`.

In [10]:
# Colonnes 100% NaN  a supprimer
cols_100 = missing_pct[missing_pct == 100.0].index.tolist()
print(f"  Colonnes 100% NaN ({len(cols_100)})  a supprimer :")
for c in cols_100:
    print(f"    {c}")

# Colonnes numeriques avec NaN  a imputer (moyenne/mediane)
num_with_nan = [c for c in missing.index if df[c].dtype in ['float64', 'int64'] and missing_pct[c] < 100]
print(f"\n Colonnes numeriques avec NaN ({len(num_with_nan)})  imputation par moyenne/mediane :")
for c in num_with_nan:
    skew = df[c].skew()
    method = 'Mediane' if abs(skew) > 1 else 'Moyenne'
    print(f"    {c:<30} ({missing_pct[c]}% NaN, skew={skew:.2f}  {method})")

# Colonnes categorielles avec NaN  a imputer (mode/valeur par defaut)
cat_with_nan = [c for c in missing.index if df[c].dtype == 'object' and missing_pct[c] < 100]
print(f"\n Colonnes categorielles avec NaN ({len(cat_with_nan)})  imputation par mode/valeur par defaut :")
for c in cat_with_nan:
    print(f"    {c:<30} ({missing_pct[c]}% NaN)")

  Colonnes 100% NaN (18)  a supprimer :
    remarques
    note_examen_national
    note_controle_continu
    note_finale_bac
    mention_bac
    probabilite_reussite
    date_mise_a_jour
    gestion_annuel
    gestion_s2
    gestion_s1
    comptabilite_annuel
    comptabilite_s2
    comptabilite_s1
    economie_annuel
    economie_s2
    economie_s1
    type_handicap
    note_examen_regional

 Colonnes numeriques avec NaN (0)  imputation par moyenne/mediane :

 Colonnes categorielles avec NaN (0)  imputation par mode/valeur par defaut :


---

##  Conclusion

Le dataset brut contient **10 000 enregistrements** et **286 colonnes**. Les principaux problemes identifies sont :

- **Decalage de colonnes** dans les dernieres colonnes (apres `performance_cible`)  corrige dans ce notebook
- **Colonnes entierement vides**  a supprimer
- **Valeurs manquantes numeriques**  a imputer par moyenne ou mediane selon l'asymetrie
- **Valeurs manquantes categorielles**  a imputer par le mode ou une valeur par defaut semantique
- **Aucun doublon** detecte

**Prochaine etape**  Executer `clean_data.ipynb` pour nettoyer les donnees.